In [51]:
import pandas as pd
import argparse
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Linear

import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData
from sklearn.preprocessing import MinMaxScaler

In [50]:
dataset = pd.read_csv('C:/Users/ruben/OneDrive/Desktop/Datasets/feature_engineering_dataset.csv')
dataset = dataset.drop(columns=['cc_num', 'merchant'], axis=1)
dataset.head()

,index,category,gender,city,state,job,is_fraud,cc_user,merchant_num,hour,day,month,weekday,age,distance_km,hours_diff_bet_trans,amt_log
0,1017,63287,0,3527,19322,2021,0,0.0,293.0,12,1,1,1,33,127.61,0,2.112635
1,2724,131659,0,3527,19322,2021,0,0.0,43.0,8,2,1,2,33,110.31,19,3.987872
2,2726,131659,0,3527,19322,2021,0,0.0,399.0,8,2,1,2,33,21.79,0,4.419804
3,2882,113035,0,3527,19322,2021,0,0.0,126.0,12,2,1,2,33,87.20,3,3.577669
4,2907,123115,0,3527,19322,2021,0,0.0,41.0,13,2,1,2,33,74.21,0,3.338613


In [52]:
# Columns to exclude from normalization
# Index, cc_user and merchant_num are unique IDs that will be used for the construction of the graph. Gender and is_fraud are binary categorical variables.
exclude_columns = ['index', 'gender', 'is_fraud', 'cc_user', 'merchant_num']

# Columns to normalize
columns_to_normalize = [col for col in dataset.columns if col not in exclude_columns]

# Normalize the dataset using MinMaxScaler
scaler = MinMaxScaler()
dataset[columns_to_normalize] = scaler.fit_transform(dataset[columns_to_normalize])

In [54]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

device = device = torch.device('cpu')
print(device)

cpu


In [55]:
data = HeteroData()

user_mapping = {idx: i for i, idx in enumerate(dataset['cc_user'].unique())}
merchant_mapping = {idx: i for i, idx in enumerate(dataset['merchant_num'].unique())}
data['user'].num_nodes = len(user_mapping)
data['merchant'].num_nodes = len(merchant_mapping)

src = [user_mapping[idx] for idx in dataset['cc_user']]
dst = [merchant_mapping[idx] for idx in dataset['merchant_num']]
edge_index = torch.tensor([src, dst])

fraud = torch.from_numpy(dataset['is_fraud'].values).to(torch.long)

data['user', 'pays', 'merchant'].edge_index = edge_index
data['user', 'pays', 'merchant'].edge_label = fraud

print(data)

HeteroData(
  user={ num_nodes=983 },
  merchant={ num_nodes=693 },
  (user, pays, merchant)={
    edge_index=[2, 1296675],
    edge_label=[1296675],
  }
)


In [56]:
# Extract unique user and merchant features using pandas DataFrames
user_features_df = dataset.drop_duplicates(subset='cc_user').set_index('cc_user').loc[:, ['job', 'gender', 'age', 'state', 'city']]
merchant_features_df = dataset.drop_duplicates(subset='merchant_num').set_index('merchant_num').loc[:, ['category']]

# Create zero-filled tensors for features
user_features_mapped = torch.zeros((len(user_mapping), user_features_df.shape[1]), dtype=torch.float)
merchant_features_mapped = torch.zeros((len(merchant_mapping), merchant_features_df.shape[1]), dtype=torch.float)

# Map features to nodes using the mappings
for user, idx in user_mapping.items():
    user_features_mapped[idx] = torch.tensor(user_features_df.loc[user].values, dtype=torch.float)

for merchant, idx in merchant_mapping.items():
    merchant_features_mapped[idx] = torch.tensor(merchant_features_df.loc[merchant].values, dtype=torch.float)

In [57]:
# data['user'].x = torch.tensor(clients.values, dtype=torch.float)
# data['merchant'].x = torch.tensor(merchants.values, dtype=torch.float)
data['user'].x = user_features_mapped
data['merchant'].x = merchant_features_mapped
del data['user'].num_nodes, data['merchant'].num_nodes

data = T.ToUndirected()(data)
del data['merchant', 'rev_pays', 'user'].edge_label  # Remove "reverse" label.

# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('user', 'pays', 'merchant')],
    rev_edge_types=[('merchant', 'rev_pays', 'user')],
)(data)

data.to(device)
print(data)

HeteroData(
  user={ x=[983, 5] },
  merchant={ x=[693, 1] },
  (user, pays, merchant)={
    edge_index=[2, 1296675],
    edge_label=[1296675],
  },
  (merchant, rev_pays, user)={ edge_index=[2, 1296675] }
)


In [59]:
data.validate()

True

In [58]:
weight = torch.bincount(train_data['user', 'merchant'].edge_label)
weight = weight.max() / weight

In [60]:
def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

In [61]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [62]:
class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['merchant'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

In [63]:
class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)


model = Model(hidden_channels=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [64]:
def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'merchant'].edge_label_index)
    target = train_data['user', 'merchant'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['user', 'merchant'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'merchant'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)

In [65]:
for epoch in range(1, 301):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

Epoch: 001, Loss: 1.0088, Train: 0.4762, Val: 0.4762, Test: 0.4762
Epoch: 002, Loss: 0.4946, Train: 0.8708, Val: 0.8709, Test: 0.8709
Epoch: 003, Loss: 0.7731, Train: 0.5567, Val: 0.5567, Test: 0.5566
Epoch: 004, Loss: 0.4994, Train: 0.2850, Val: 0.2851, Test: 0.2850
Epoch: 005, Loss: 0.5873, Train: 0.2425, Val: 0.2426, Test: 0.2425
Epoch: 006, Loss: 0.6302, Train: 0.3214, Val: 0.3214, Test: 0.3214
Epoch: 007, Loss: 0.5571, Train: 0.4590, Val: 0.4591, Test: 0.4590
Epoch: 008, Loss: 0.4950, Train: 0.5919, Val: 0.5920, Test: 0.5920
Epoch: 009, Loss: 0.5096, Train: 0.6603, Val: 0.6604, Test: 0.6604
Epoch: 010, Loss: 0.5450, Train: 0.6441, Val: 0.6441, Test: 0.6441
Epoch: 011, Loss: 0.5345, Train: 0.5754, Val: 0.5755, Test: 0.5755
Epoch: 012, Loss: 0.5030, Train: 0.4975, Val: 0.4976, Test: 0.4976
Epoch: 013, Loss: 0.4904, Train: 0.4359, Val: 0.4360, Test: 0.4359
Epoch: 014, Loss: 0.4978, Train: 0.4001, Val: 0.4002, Test: 0.4002
Epoch: 015, Loss: 0.5090, Train: 0.3912, Val: 0.3913, Test: 0.

KeyboardInterrupt: 